In [18]:
import pandas as pd
import numpy as np
%config InlineBackend.figure_format = 'retina'
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import scipy
from matplotlib.colors import ListedColormap
from functools import partial

In [19]:
df = pd.read_csv('wine.data', header=None, delimiter=',', dtype=float)
print(df)

      0      1     2     3     4      5     6     7     8     9      10    11  \
0    1.0  14.23  1.71  2.43  15.6  127.0  2.80  3.06  0.28  2.29   5.64  1.04   
1    1.0  13.20  1.78  2.14  11.2  100.0  2.65  2.76  0.26  1.28   4.38  1.05   
2    1.0  13.16  2.36  2.67  18.6  101.0  2.80  3.24  0.30  2.81   5.68  1.03   
3    1.0  14.37  1.95  2.50  16.8  113.0  3.85  3.49  0.24  2.18   7.80  0.86   
4    1.0  13.24  2.59  2.87  21.0  118.0  2.80  2.69  0.39  1.82   4.32  1.04   
..   ...    ...   ...   ...   ...    ...   ...   ...   ...   ...    ...   ...   
173  3.0  13.71  5.65  2.45  20.5   95.0  1.68  0.61  0.52  1.06   7.70  0.64   
174  3.0  13.40  3.91  2.48  23.0  102.0  1.80  0.75  0.43  1.41   7.30  0.70   
175  3.0  13.27  4.28  2.26  20.0  120.0  1.59  0.69  0.43  1.35  10.20  0.59   
176  3.0  13.17  2.59  2.37  20.0  120.0  1.65  0.68  0.53  1.46   9.30  0.60   
177  3.0  14.13  4.10  2.74  24.5   96.0  2.05  0.76  0.56  1.35   9.20  0.61   

       12      13  
0    3.

In [20]:
df = df.drop([2,3,4,5,6,8,9,10,11,12,13], axis=1)
df = df[df[0] != 3.0]

In [21]:
df[0] = df[0].replace({1.0: 1, 2.0: -1})
df[0] = df[0].astype(float)
df

,0,1,7
0,1.0,14.23,3.06
1,1.0,13.20,2.76
2,1.0,13.16,3.24
3,1.0,14.37,3.49
4,1.0,13.24,2.69
...,...,...,...
125,-1.0,12.07,2.65
126,-1.0,12.43,3.15
127,-1.0,11.79,2.24
128,-1.0,12.37,2.45


In [22]:
X = df.iloc[:, 1:3].values
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X) 
Y = df.iloc[:, 0].values    
Y = Y.reshape(-1, 1).astype(float)  
X_and_Y = np.hstack((X, Y))     
np.random.seed(1)              
np.random.shuffle(X_and_Y)     
print(X.shape)
print(Y.shape)
print(X_and_Y[0])

(130, 2)
(130, 1)
[-0.81784764 -0.61230964 -1.        ]


# Logistic

# 20 training

In [23]:
# Divide the data points into training set and test set.
X_shuffled = X_and_Y[:,:2]
Y_shuffled = X_and_Y[:,2]

split_index = int(0.2 * len(X_shuffled)) 

X_train = X_shuffled[:split_index]
Y_train = Y_shuffled[:split_index]
X_test = X_shuffled[split_index:]
Y_test = Y_shuffled[split_index:]   
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(26, 2)
(26,)
(104, 2)
(104,)


In [24]:
# Sigmoid function: sigmoid(z) = 1/(1 + e^(-z))
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))
# Judge function: 1(a != b).
def judge(a, b):
    """
    Judge function: 1(a != b).
    Return 1 if a != b, otherwise return 0.
    """
    if a != b:
        return 1
    else:
        return 0
    
def f_logistic(x, W, b):
    """
    Logistic classifier: f(x, W, b)
    This function should return -1 or 1.

    x should be a 2-dimensional vector, 
    W should be a 2-dimensional vector,
    b should be a scalar.
    """
    if sigmoid(W.T.dot(x)+b) >= 0.5:
        return 1
    else:
        return -1
    
# Calculate error given feature vectors X and labels Y.
def calc_error(X, Y, W, b):
    e = 0
    n = len(X)
    for (xi, yi) in zip(X, Y):
        # Hint: Use judge() and f_logistic()
        predicted_label = f_logistic(xi, W, b)
        e+=judge(yi, predicted_label)
    
    # Hint: remember we want the average error.
    e = 1.0*e/n
    return e
# Gradient of L(W, b) with respect to W and b.
def grad_L_W_b(X, Y, W, b):
    P = sigmoid(Y*(np.dot(X, W)+b))
    delta = np.ones_like(Y)-P
    grad_W = -np.dot(X.T,delta*Y)
    grad_b = -np.dot(np.ones(delta.shape).T, delta * Y) #dotting ones is the same as summing everything, wow
    return grad_W, grad_b
# Loss L(W, b).
def L_W_b(X, Y, W, b):
    P = sigmoid(Y*(np.dot(X, W)+b))
    loss = -np.dot(np.ones_like(Y).T, np.log(P))

    return loss
def logistic_regression(X_train, Y_train):
    # Some settings.
    losses = []           # Error history.
    learning_rate = 0.001 # Learning rate, fixed
    iterations    = 10000 # Iteration number, fixed

    # Gradient descent algorithm for logistic regression.
    # Step 1. Initialize the parameters W, b.
    W      = np.zeros(2)  # Weight.
    b      = 0.0          # Bias.

    # Logistic regression learning algorithm.
    for i in range(iterations):
        # Step 2. Compute the partial derivatives.
        grad_W, grad_b = grad_L_W_b(X_train, Y_train, W, b)
        # Step 3. Update the parameters.
        W = W-learning_rate*grad_W
        b = b-learning_rate*grad_b
        
        # Track the training losses.
        loss = L_W_b(X_train, Y_train, W, b)
        losses.append(loss)

    return W, b, losses
#W,b = logistic_regression(X_train, Y_train)
def cross_validate_logistic_regression(X, Y, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    validation_errors = []  # To store validation errors for each fold
    models = []  # To store W, b, and losses for each fold

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        Y_train, Y_val = Y[train_idx], Y[val_idx]

        # Train the model on the training data
        W, b, losses = logistic_regression(X_train, Y_train)
        
        # Evaluate on the validation set
        error = calc_error(X_val, Y_val, W, b)
        validation_errors.append(error)
        
        # Store the model for this fold
        models.append((W, b, losses))
    
    # Average validation error across folds
    avg_validation_error = np.mean(validation_errors)
    return avg_validation_error, models, validation_errors


In [25]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
C_values = [0.001, 0.01, 0.1, 1, 10, 100]

best_C = None
best_score = 0
#Cross validation
for C in C_values:
    model = LogisticRegression(C=C, solver='lbfgs')
    scores = cross_val_score(model, X_train, Y_train, cv=5)  # 5-fold cross-validation
    avg_score = scores.mean()
    
    print(f"C: {C}, Average Cross-Validation Score: {avg_score}")

    if avg_score > best_score:
        best_score = avg_score
        best_C = C

print(f"Best C: {best_C} with score: {best_score}")
from sklearn.metrics import accuracy_score
final_model = LogisticRegression(C=best_C, solver='lbfgs')
final_model.fit(X_train, Y_train)
Y_train_pred = final_model.predict(X_train)
training_accuracy = accuracy_score(Y_train, Y_train_pred)
training_error = 1 - training_accuracy
print(f"Training accuracy: {training_accuracy}")
print(f"Training error: {training_error}")
Y_test_pred = final_model.predict(X_test)
test_accuracy = accuracy_score(Y_test, Y_test_pred)
test_error = 1 - test_accuracy
print(f"Test accuracy: {test_accuracy}")
print(f"Test error: {test_error}")

C: 0.001, Average Cross-Validation Score: 0.5199999999999999
C: 0.01, Average Cross-Validation Score: 0.7200000000000001
C: 0.1, Average Cross-Validation Score: 0.96
C: 1, Average Cross-Validation Score: 0.9199999999999999
C: 10, Average Cross-Validation Score: 0.9199999999999999
C: 100, Average Cross-Validation Score: 0.9199999999999999
Best C: 0.1 with score: 0.96
Training accuracy: 0.9230769230769231
Training error: 0.07692307692307687
Test accuracy: 0.9230769230769231
Test error: 0.07692307692307687


# 50 training

In [26]:
# Divide the data points into training set and test set.
X_shuffled = X_and_Y[:,:2]
Y_shuffled = X_and_Y[:,2]

split_index = int(0.5 * len(X_shuffled))

X_train = X_shuffled[:split_index]
Y_train = Y_shuffled[:split_index]
X_test = X_shuffled[split_index:]
Y_test = Y_shuffled[split_index:]   
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(65, 2)
(65,)
(65, 2)
(65,)


In [27]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
C_values = [0.001, 0.01, 0.1, 1, 10, 100]

best_C = None
best_score = 0
#Cross validation
for C in C_values:
    model = LogisticRegression(C=C, solver='lbfgs')
    scores = cross_val_score(model, X_train, Y_train, cv=5)  # 5-fold cross-validation
    avg_score = scores.mean()
    
    print(f"C: {C}, Average Cross-Validation Score: {avg_score}")

    if avg_score > best_score:
        best_score = avg_score
        best_C = C

print(f"Best C: {best_C} with score: {best_score}")
from sklearn.metrics import accuracy_score
final_model = LogisticRegression(C=best_C, solver='lbfgs')
final_model.fit(X_train, Y_train)
Y_train_pred = final_model.predict(X_train)
training_accuracy = accuracy_score(Y_train, Y_train_pred)
training_error = 1 - training_accuracy
print(f"Training accuracy: {training_accuracy}")
print(f"Training error: {training_error}")
Y_test_pred = final_model.predict(X_test)
test_accuracy = accuracy_score(Y_test, Y_test_pred)
test_error = 1 - test_accuracy
print(f"Test accuracy: {test_accuracy}")
print(f"Test error: {test_error}")

C: 0.001, Average Cross-Validation Score: 0.5692307692307692
C: 0.01, Average Cross-Validation Score: 0.6923076923076923
C: 0.1, Average Cross-Validation Score: 0.876923076923077
C: 1, Average Cross-Validation Score: 0.8923076923076924
C: 10, Average Cross-Validation Score: 0.876923076923077
C: 100, Average Cross-Validation Score: 0.876923076923077
Best C: 1 with score: 0.8923076923076924
Training accuracy: 0.9230769230769231
Training error: 0.07692307692307687
Test accuracy: 0.9384615384615385
Test error: 0.06153846153846154


# 80 training

In [28]:
# Divide the data points into training set and test set.
X_shuffled = X_and_Y[:,:2]
Y_shuffled = X_and_Y[:,2]

split_index = int(0.8 * len(X_shuffled))

X_train = X_shuffled[:split_index]
Y_train = Y_shuffled[:split_index]
X_test = X_shuffled[split_index:]
Y_test = Y_shuffled[split_index:]   
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(104, 2)
(104,)
(26, 2)
(26,)


In [29]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
C_values = [0.001, 0.01, 0.1, 1, 10, 100]

best_C = None
best_score = 0
#Cross validation
for C in C_values:
    model = LogisticRegression(C=C, solver='lbfgs')
    scores = cross_val_score(model, X_train, Y_train, cv=5)  # 5-fold cross-validation
    avg_score = scores.mean()
    
    print(f"C: {C}, Average Cross-Validation Score: {avg_score}")

    if avg_score > best_score:
        best_score = avg_score
        best_C = C

print(f"Best C: {best_C} with score: {best_score}")
from sklearn.metrics import accuracy_score
final_model = LogisticRegression(C=best_C, solver='lbfgs')
final_model.fit(X_train, Y_train)
Y_train_pred = final_model.predict(X_train)
training_accuracy = accuracy_score(Y_train, Y_train_pred)
training_error = 1 - training_accuracy
print(f"Training accuracy: {training_accuracy}")
print(f"Training error: {training_error}")
Y_test_pred = final_model.predict(X_test)
test_accuracy = accuracy_score(Y_test, Y_test_pred)
test_error = 1 - test_accuracy
print(f"Test accuracy: {test_accuracy}")
print(f"Test error: {test_error}")

C: 0.001, Average Cross-Validation Score: 0.5671428571428571
C: 0.01, Average Cross-Validation Score: 0.8571428571428571
C: 0.1, Average Cross-Validation Score: 0.9047619047619048
C: 1, Average Cross-Validation Score: 0.9333333333333333
C: 10, Average Cross-Validation Score: 0.9238095238095239
C: 100, Average Cross-Validation Score: 0.9238095238095239
Best C: 1 with score: 0.9333333333333333
Training accuracy: 0.9423076923076923
Training error: 0.05769230769230771
Test accuracy: 0.8846153846153846
Test error: 0.11538461538461542
